# Instanciar Spark

In [6]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col, from_json , explode, expr, lit, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy.dialects.postgresql import insert
import psycopg2


spark_session = SparkSession.builder \
    .appName("Ibge_silver") \
    .config("spark.master", "local[*]") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.executor.instances", "3") \
    .getOrCreate()

# PANDAS 


In [3]:
def call_upsert(table, conn, keys, data_iter):

    data = [dict(zip(keys, row)) for row in data_iter]

    insert_statement = insert(table.table).values(data)
    upsert_statement = insert_statement.on_conflict_do_update(
        constraint=f"{table.table.name}_pkey",
        set_={c.key: c for c in insert_statement.excluded},
    )

    conn.execute(upsert_statement)

# SQL CONNECTION

In [4]:
# Configurações de conexão com o PostgreSQL
# Este item é totalmente editavel, para uma conexão de sua preferencia (Vamos fazer conforme o docker-compose que subimos)

host = 'localhost'
port = '8085'

gold_url = f"jdbc:postgresql://{host}:{port}/gold_data"
silver_url = f"jdbc:postgresql://{host}:{port}/silver_data"
properties = {"user": "ibge", "password": "ibge", "driver": "org.postgresql.Driver"}

# Criar um engine do SQLAlchemy
gold_engine = create_engine(f"postgresql://{properties['user']}:{properties['password']}@{host}:{port}/gold_data")
silver_engine = create_engine(f"postgresql://{properties['user']}:{properties['password']}@{host}:{port}/silver_data")



# REIGAO

In [4]:
# INSERÇÃO DA DIMENSÃO REGIÃO USANDO PANDAS

query = 'SELECT * FROM public.regiao'

for data_frame in pd.read_sql(query,silver_engine,chunksize=10000):
    try:
        pd.DataFrame(data_frame).to_sql(
            'regiao', 
            gold_engine, 
            schema='public', 
            index=False, 
            if_exists="append", 
            method=call_upsert
        )
        print("UpInsert executado")
    except Exception as e :
        print(f'Exceção {str(e)}')            

UpInsert executado


In [5]:
# INSERÇÃO DA DIMENSÃO ESTADO USANDO PANDAS

query = """
select
	*
from
	public.estado"""

for data_frame in pd.read_sql(query,silver_engine,chunksize=10000):
    try:
        pd.DataFrame(data_frame).to_sql(
            'estado', 
            gold_engine, 
            schema='public', 
            index=False, 
            if_exists="append", 
            method=call_upsert
        )
        print("UpInsert executado")
    except Exception as e :
        print(f'Exceção {str(e)}')      

UpInsert executado


In [7]:
# INSERÇÃO DA DIMENSÃO ESTADO USANDO PANDAS

sql_municipio = """
    select
	id_municipio,
	municipio,
	uf,
	flag_capital
from
	public.municipio;
"""
for data_frame in pd.read_sql(sql_municipio,silver_engine,chunksize=10000):
    try:
        pd.DataFrame(data_frame).to_sql(
            'municipio', 
            gold_engine, 
            schema='public', 
            index=False, 
            if_exists="append", 
            method=call_upsert
        )
        print("UpInsert executado")
    except Exception as e :
        print(f'Exceção {str(e)}')   


UpInsert executado


In [25]:
# lat e long

coordenadas = {
	'AC': [-8.77, -70.55],
	'AL': [-9.62, -36.82],
	'AM': [-3.47, -65.10],
	'AP': [1.41, -51.77],
	'BA': [-13.29, -41.71],
	'CE': [-5.20, -39.53],
	'DF': [-15.83, -47.86],
	'ES': [-19.19, -40.34],
	'GO': [-15.98, -49.86],
	'MA': [-5.42, -45.44],
	'MT': [-12.64, -55.42],
	'MS': [-20.51, -54.54],
	'MG': [-18.10, -44.38],
	'PA': [-3.79, -52.48],
	'PB': [-7.28, -36.72],
	'PR': [-24.89, -51.55],
	'PE': [-8.38, -37.86],
	'PI': [-6.60, -42.28],
	'RJ': [-22.25, -42.66],
	'RN': [-5.81, -36.59],
	'RO': [-10.83, -63.34],
	'RS': [-30.17, -53.50],
	'RR': [1.99, -61.33],
	'SC': [-27.45, -50.95],
	'SE': [-10.57, -37.45],
	'SP': [-22.19, -48.79],
	'TO': [-9.46, -48.26]
}

df = pd.DataFrame(list(coordenadas.items()), columns=['UF', 'Coords'])
df[['lat', 'long']] = pd.DataFrame(df['Coords'].tolist(), index=df.index)

# Descartar a coluna original "Coords"
df = df.drop('Coords', axis=1)

df.to_sql(
            'lat_long', 
            gold_engine, 
            schema='public', 
            index=False, 
            if_exists="replace"
        )



27

# PESQUISAS (leitura)

In [14]:
# IDH

# Definir a consulta SQL
sql_query = """
    select
	ano,
	municipio,
	tp.pesquisa  as pesquisa ,
	valor
from
	public.pesquisas p
left join
	tipo_pesquisa tp 
on
	p.cd_pesquisa  = tp.cd_pesquisa 
where
	p.ano in (2018,2019,2020,2021)
"""

# Ler os dados do banco de dados usando Spark
df_pesquisa = spark_session.read.jdbc(url=silver_url, table=f"({sql_query}) as subquery", properties=properties)

df_pivotado = df_pesquisa.groupBy("ano", "municipio") \
    .pivot("pesquisa") \
    .agg({"valor": "first"})

# df_pivotado.show()

df_pesquisa_final = df_pivotado.select(
    col('ano'),
    col('municipio'),
    col('idh').astype(DoubleType()),
    col('pib').astype(DoubleType()),
    col('populacional').astype(IntegerType())
)

# Calcular quartis da coluna de população
quartis = df_pesquisa_final.approxQuantile("populacional", [0.25, 0.5, 0.75], 0.01)

# Definir os limites dos quadrantes com base nos quartis
limite_quadrante_1 = quartis[0]
limite_quadrante_2 = quartis[1]
limite_quadrante_3 = quartis[2]
limite_quadrante_4 = float("inf")  # O último quadrante vai até o infinito

# Adicionar uma coluna "Faixa de População" ao DataFrame com base nos limites dos quadrantes
df_pesquisa_final = df_pesquisa_final.withColumn("Faixa de População",
                                                when(col("populacional") <= limite_quadrante_1, f"Entre 0 e {int(limite_quadrante_1)}")
                                                .when((col("populacional") > limite_quadrante_1) & (col("populacional") <= limite_quadrante_2), f"Entre {int(limite_quadrante_1)} e {int(limite_quadrante_2)}")
                                                .when((col("populacional") > limite_quadrante_2) & (col("populacional") <= limite_quadrante_3), f"Entre {int(limite_quadrante_2)} e {int(limite_quadrante_3)}")
                                                .when((col("populacional") > limite_quadrante_3) & (col("populacional") <= limite_quadrante_4), f"Maior que {int(limite_quadrante_3)}")
                                                .otherwise("Outro"))

# Exibir o DataFrame resultante
df_pesquisa_final.show()


df_pesquisa_final.write.format("jdbc").option("url", gold_url) \
.option("user", "ibge") \
.option("password", "ibge") \
.option("dbtable", "public.fato_pesquisa") \
.mode('overwrite') \
.save()



+----+---------+----+----------+------------+-------------------+
| ano|municipio| idh|       pib|populacional| Faixa de População|
+----+---------+----+----------+------------+-------------------+
|2018|  1100015|NULL|  498980.0|       23167|Entre 11414 e 24725|
|2018|  1100023|NULL| 2464704.0|      106168|    Maior que 24725|
|2018|  1100031|NULL|  140503.0|        5438| Entre 5366 e 11414|
|2018|  1100049|NULL| 2175840.0|       84813|    Maior que 24725|
|2018|  1100056|NULL|  470647.0|       16444|Entre 11414 e 24725|
|2018|  1100064|NULL|  330232.0|       16227|Entre 11414 e 24725|
|2018|  1100072|NULL|  320416.0|        7567| Entre 5366 e 11414|
|2018|  1100080|NULL|  230151.0|       17855|Entre 11414 e 24725|
|2018|  1100098|NULL|  606072.0|       32047|    Maior que 24725|
|2018|  1100106|NULL|  837459.0|       45783|    Maior que 24725|
|2018|  1100114|NULL| 1484555.0|       51933|    Maior que 24725|
|2018|  1100122|NULL| 3466810.0|      127907|    Maior que 24725|
|2018|  11